In [1]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [20]:
torch.manual_seed(42)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [22]:
df = pd.read_csv('fashion-mnist_test.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.shape

(10000, 785)

In [24]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [25]:
type(X)

numpy.ndarray

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X_train = X_train/255.0
X_test = X_test/255.0

In [28]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]


In [29]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [32]:
class MyNN(nn.Module):

  def __init__(self, input_features):
    super().__init__()

    self.features = nn.Sequential(
      nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(32, 64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.classifier = nn.Sequential(
      nn.Flatten(),
      
      nn.Linear(64*7*7, 128),
      nn.ReLU(),
      nn.Dropout(p=0.4),

      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Dropout(p=0.4),

      nn.Linear(64, 10)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.classifier(x)

    return x

In [33]:
learning_rate = 0.1
epochs = 100

In [34]:
model = MyNN(1)

model = model.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [35]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)

    loss = loss_function(y_pred, batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

Epoch: 1, Loss: 0.8839643436670304
Epoch: 2, Loss: 0.5756397651135922
Epoch: 3, Loss: 0.5016718496084214
Epoch: 4, Loss: 0.4299055549800396
Epoch: 5, Loss: 0.39596491256356237
Epoch: 6, Loss: 0.3802167035639286
Epoch: 7, Loss: 0.34088245233893394
Epoch: 8, Loss: 0.3177663570046425
Epoch: 9, Loss: 0.29483964598178863
Epoch: 10, Loss: 0.2877407397031784
Epoch: 11, Loss: 0.2827796341404319
Epoch: 12, Loss: 0.25672291004657743
Epoch: 13, Loss: 0.23317693753540517
Epoch: 14, Loss: 0.2203387898914516
Epoch: 15, Loss: 0.21201797268539668
Epoch: 16, Loss: 0.1993758711516857
Epoch: 17, Loss: 0.17184807418286802
Epoch: 18, Loss: 0.16554691026546062
Epoch: 19, Loss: 0.1603579935245216
Epoch: 20, Loss: 0.1695531075093895
Epoch: 21, Loss: 0.14867502545099706
Epoch: 22, Loss: 0.1516279066298157
Epoch: 23, Loss: 0.13901346877776086
Epoch: 24, Loss: 0.13272420876659452
Epoch: 25, Loss: 0.14610681676864623
Epoch: 26, Loss: 0.11138976755412296
Epoch: 27, Loss: 0.12526828312873842
Epoch: 28, Loss: 0.1012

In [36]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [37]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')

Accuracy: 89.75


In [38]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')

Accuracy: 99.9875
